First is necessary to know which files are going to be used in the data set, in our case we tried to use all the information avaiable (so we should have 1012 patient id's inside our local copy of the LIDC-IDRI) so that we could get more accurate results and since we would be using "one" image from each pacient "nodule". Also we need the Patients ID's so that we can have access to the scan's, conveniently the names of the folders inside the LIDC-IDRI correspond to the Patient ID 

In [32]:
import os

#specify the path to your dataset where you have the folders with the name of the pacients
directory="data/dataset/LIDC-IDRI/"
#list all the folders inside our local copy of LIDC-IDRI (that correspond to the patient id's that are going to be used to get the scans and annotations)
folders=os.listdir(directory)

print(f"Number of patients in the dataset: {len(folders)}")
#just so that it is easier to know which patients we are don't have in our copy of the LIDC-IDRI
folders.sort()
print()
#printing the patients id's that we couldn't download
print("List of the id's that are missing from our dataset:")
count=1
for folder in folders:
    if(str(count) not in folder):
        print(folder)
        count=int(folder.split('-')[-1])
    count+=1


Number of patients in the dataset: 1010

List of the id's that are missing from our dataset:
LIDC-IDRI-0239
LIDC-IDRI-0586


As it is possible to see the number is diferent from the expected (1012 number of people that are avaiable in the original LODC-IRDI), this is due to the fact that during the download (which was made using the offical tool nbia-data-retriver) we did not have authority to download this instances, one reason could be the patient's own choice, since this number is only the possible data loss is should not impact our conclusions 

In [ ]:
import pylidc as pl

n_lesions=0
scans_0_nods=[]

for folder in folders:
    pid=folder

    #get the scann corresponding to the patient_id that we have 
    scan=pl.query(pl.Scan).filter(pl.Scan.patient_id==pid).first()
    #get all the nodules annotations, each list corresponds to the same nodules
    nods=scan.cluster_annotations()    
    n_lesions+=len(nods)
    if(len(nods)==0):
        scans_0_nods.append([scan.patient_id,scan.annotations])
        


print(f"Number of lesions: {n_lesions}" )
print(f"Number of scanns with 0 nodules: {len(scans_0_nods)}")
#print(scans_0_nods)

After executing the code above the output should be:

Number of lesions: 2625                                                                                                                                                  
Number of scanns with 0 annotations: 135 (13 of these gave an error in the clustering to the nearest nodule so they have annotations but we would have to adjust them to the manually to one of the nodes present)

Note: The output of the cell was cleared because of the error message of the 13 scanns mentioned, jsut so the notebook wouldn't be clutterd

This code only serves to give us the numbers of rows that ours datasets should have if everything runs without errors, and how many more scans are not going to used.This is because if an scan doesn't have annotations we are incappable of knowing many informations that are required for it to be part of the dataset (for example the malignancy)

This function simply gets the malignancy of an nodule.It is a crucial part of our project since it is this malignancy value that is going to be used as a Label for our classes. Using pylidc we can get this value easily since for each scann we have access to it's annotations, and in each annotation we have one value for malignancy given by an radiologist. In order to get an more precise value for the malignancy we calculate the mean between the diferent annotations, since getting that value from a single annotation could introduce the bias of that radiologist 

In [3]:
def get_malignancy(anns):
    #Get the Malignancy of the lesion
    #
    #   1-‘Highly Unlikely’
    #   2-‘Moderately Unlikely’
    #   3-‘Indeterminate’
    #   4-‘Moderately Suspicious’
    #   5-‘Highly Suspicious’
    #this feature will be used as a label to the dataset
    malignancy=0
    for i in range(len(anns)):
        malignancy+=anns[i].malignancy
    malignancy/=len(anns)
    malignancy=round(malignancy)

    return malignancy

Same as the get_malignancy function it also uses annotations and the pylidc library to get the values of calcification, once again we calculate the mean to get a value that better represents the reality. Getting the calcification of the nodule is beneficial for our model since it indicates a high probability that the lesion is benign, it's not always the case but it is a pointer

In [11]:
def get_calcification(anns):
    #Get the pattern of calcification if present
    # 
    #   1-‘Popcorn’
    #   2-‘Laminated’
    #   3-‘Solid’
    #   4-‘Non-central’
    #   5-‘Central’
    #   6-‘Absent’
    calcification=0
    for i in range(len(anns)):
        calcification+=anns[i].calcification
    calcification/=len(anns)
    calcification=round(calcification)
    return calcification

The function get_lobulation works exactly as the previous but it gets the lobutation value. It can give an us an idea of the growth rates of an nodules, which can be indicators of malignancy

In [12]:
def get_lobulation(anns):
    #Get the degree of lobulationof the lesion
    #
    #   1-‘No Lobulation’
    #   2-‘Nearly No Lobulation’
    #   3-‘Medium Lobulation’
    #   4-‘Near Marked Lobulation’
    #   5-‘Marked Lobulation’

    lobulation=0
    for i in range(len(anns)):
        lobulation+=anns[i].lobulation
    lobulation/=len(anns)
    lobulation=round(lobulation)
    return lobulation

Similarily to lobulation spiculation is an indicator of growth rates, it is normaly associated with irregular and "spiky" margins, indicating uneven growth rates which are one of the main poiter to the malignancy of a nodule

In [13]:
def get_spiculation(anns):
    #Get the extent of spiculation present in the lesion
    #
    #   ‘No Spiculation’
    #   ‘Nearly No Spiculation’
    #   ‘Medium Spiculation’
    #   ‘Near Marked Spiculation’
    #   ‘Marked Spiculation’
    spiculation=0
    for i in range(len(anns)):
        spiculation+=anns[i].spiculation
    spiculation/=len(anns)
    spiculation=round(spiculation)
    return spiculation

The internal Texture of an nodule can gives us informtation that goes beyond the geometric form of the nodule, as a result we decided to also include it in our analyse

In [14]:
def get_internal_texture(anns):
    #Get the internal texture of the lesion
    #
    #    1-‘Non-Solid/GGO’
    #    2-‘Non-Solid/Mixed’
    #    3-‘Part Solid/Mixed’
    #    4-‘Solid/Mixed’
    #    5-‘Solid’
    texture=0
    for i in range(len(anns)):
        texture+=anns[i].texture
    texture/=len(anns)
    texture=round(texture)
    return texture

Simple features that give an idea of the size of the node, even if we know previously that since it has annotation it has to be >3mm being more precise with the size can be usefull for our model 

In [15]:
def get_diameter_surface_area_volume(anns):
    #Get the mean value of all anotations for this lesion of the diameter, surface_area, volume
    diameter=0
    surface_area=0
    volume=0
    for i in range(len(anns)):
        diameter+=anns[i].diameter
        surface_area+=anns[i].surface_area
        volume+=anns[i].volume
    diameter/=len(anns)
    surface_area/=len(anns)
    volume/=len(anns)

    return diameter,surface_area,volume

Also gives an idea of size, but this time it ins't as regularized, since it is not in an any metric unit. It could also capture diference in machines, all the images are the same resolution, which should mean that if we have the same pixeis inside an nodule they should have the same volume, but since in the LIDC-IDRI various diferent machines were used that dirence could mean that the volumes are not equal. We calculate this value using an mask calculated via concensus of all the masks created by the radilogist who made annotations for that nodule

In [16]:
import numpy as np

def get_pixeis_in(cmask):
    #Calculating the number of pixeis inside a lesion
    return np.sum(cmask)

In this function we calculate the overall circularity of the nodule acrros all of the slices, this is done onde more with the concensus mask. Circularity is an important attribute to capture for the same reasons of the lobulation and spiculation, it can indicate an uneven growth rate

In [26]:
from skimage.filters import sobel

def get_circularity(cmask):
    # Calculate circularity
    # the closer to 1 the closer to a perfect circle
    circularity=0
    for i in range(cmask.shape[-1]):
        area_k_slice_pixels=np.sum(cmask[:,:,i])#this way we get the number of pixeis of the k
        # Calculating the perimeter(in pixels) by appling an edge filter to the mask
        edges=sobel(cmask)
        perimeter_piexels=np.sum(edges[:,:,i])#so that we get the perimeter of the slice i of the lesion
        if(perimeter_piexels==0):
            circularity+=0
        else:
            circularity += (4 * np.pi * area_k_slice_pixels) / (perimeter_piexels ** 2)
    circularity/=cmask.shape[-1]

    return circularity

All of the following functions: get_median_  |  get_std_  |  get_var_  |  get_mean
Use HU to get information about radiodensity of diferent tissues hopefully to understand what kind of tissues are insede the nodule. They are canculate since using pylidc we can get the HU of the scan, then we crop the image to the relevant part to the nodule being analyzed

In [18]:
def get_median_hu(nodule_hu_values):
    #Calculate the median HU inside of the lesion
    #vol[ccbox][cmask] represents the HU inside the lesion
    return np.median(nodule_hu_values)

In [19]:
def get_std_hu(nodule_hu_values):
    #Calculate the the standard variation of HU inside the lesion
    return np.std(nodule_hu_values)

In [20]:
def get_var_hu(nodule_hu_values):
    #Calculate the variance of HU inside the lesion
    return np.var(nodule_hu_values)

In [21]:
def get_mean_hu(nodule_hu_values):
    #Calculate the mean of HU inside the lesion
    return np.mean(nodule_hu_values)

The code bellow is the onde responsible for getting all the information for each nodule. It gets Patient ID associated to an folder gets the scan and annotations of that patient and then it calculates all the features discussed above. For each nodule it also uses the cnn VG16 to try and extract even more features, we chose the VG16 because of the simple architecture and the fact that is pre-trained using the the weights of the imagenet dataset which is one of largest images datasets avaiable, although it isn't trainned specifically for medic imaging it was shown to have an good performance. We also try to use Global Average Pooling in an attempt to make the impact of the "curse of dimentionality" not be so severe. Also because of that, the original idea of filtering the original image of the nodule with an entropy and sobel filter to then fedd the VG16 was scrapped, as it would not only give us even more features that could redundant and would make the computational cost of this step even higher 
Note:This cell of code averages an run time of 80 min with an laptop pluged in, if you want to run it we advise to have the computer not be running of battery as it will increase the run time to 120 min

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from pylidc.utils import consensus
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model

#load pre-trained model withput the top layers
base_model=VGG16(weights='imagenet',include_top=False, input_shape=(512,512,3))

#add global pooling to the method
x=base_model.output
x=GlobalAveragePooling2D()(x)
cnn_model=Model(inputs=base_model.input,outputs=x)


#create the DataFrame where we ccn features are going to be stored

#create the columns names
columns_name=["PatientID",
              "Calcification",
              "Lobulation",
              "Spiculation",
              "Internal Texture",
              "Diameter (mm)",
              "Surface Area (mm^2)",
              "Volume (mm^3)",
              "Pixeis_in_lesion",
              "Circularity",
              "Median_HU_in_lesion",
              "STD_HU_in_lesion",
              "VAR_HU_in_lession",
              "Mean_HU_in_lesion"
              ]
for i in range(512):
    columns_name.append("cnn_feature "+str(i))

columns_name.append("Label")

df=pd.DataFrame(columns=columns_name)


for folder in folders:
    
    pid=folder
    print(f"Currently in scan: {pid}")

    #get the scann corresponding to the patient_id that we have 
    scan=pl.query(pl.Scan).filter(pl.Scan.patient_id==pid).first()
    
    #get all the nodules annotations, each list corresponds to the same lesions 
    nods=scan.cluster_annotations()
    #nods is a list of list of annotations agrouped by nodule

    #create an volume with all of the dicom images of that scan (basicly an 3D image of the CT)
    vol=scan.to_volume()

    padding=[(30,20),(10,25),(0,0)]
    #anns represents all the annotations for one lesion in this scann
    #nods represents a list of all the annotations for each lesion in this scan 
    for anns in nods:
        #get the concensus for the contours of this nodule
        cmask,cbbox,masks=consensus(anns,clevel=0.5,pad=padding)

        #get the central slice of the computed bounding box
        k=int(0.5*(cbbox[2].stop-cbbox[2].start))

        img=vol[cbbox][:,:,k]


        #applying possible usefull filters since we don't have that many lesions (2625)
        from skimage.filters.rank import entropy
        from skimage.morphology import disk
        from skimage.util import img_as_ubyte
        entropy_img=entropy(img_as_ubyte(img),disk(1))


        from skimage.filters import sobel
        sobel_img=sobel(img)

        #getting the images ready for the cnn
        imgs=[img,entropy_img,sobel_img]

        for i in range(1):#we can change this after if we want to use the filters
            #making each image have 3 dimensions
            imgs[i] = np.expand_dims(imgs[i], axis=-1)

            #resizing the images so that they match the input of the cnn
            imgs[i]=tf.image.resize(imgs[i],(512,512))

            #converting the images to rgb
            if(imgs[i].shape[-1]==1):
                imgs[i]=tf.image.grayscale_to_rgb(imgs[i])
            
            #preprocess to the VGG16
            imgs[i]=preprocess_input(imgs[i].numpy())

            #get the features learned by the cnn
            features=cnn_model(np.expand_dims(imgs[i], axis=0))
            features=features.numpy().flatten()

            #create the row with everything that is going in the dataset 
            row=[pid]
            row.append(get_calcification(anns))
            row.append(get_lobulation(anns))
            row.append(get_spiculation(anns))
            row.append(get_internal_texture(anns))
            row.extend(get_diameter_surface_area_volume(anns))
            row.append(get_pixeis_in(cmask))
            row.append(get_circularity(cmask))
            row.append(get_median_hu(vol[cbbox][cmask]))
            row.append(get_std_hu(vol[cbbox][cmask]))
            row.append(get_var_hu(vol[cbbox][cmask]))
            row.append(get_mean_hu(vol[cbbox][cmask]))
            #this are the cnn_extracted features
            for x in features:
                row.append(x)
            #this is the label
            row.append(get_malignancy(anns))
            df.loc[len(df.index)]=row
        
    #     break #this breaks are for when debugging not having to run all the scanns
    # break #this breaks are for when debugging not having to run all the scanns


Store the current cnn features values to make it possible to make use them without having to calculate them again as it can take a while (93 min first time we tried)


In [29]:
df.to_csv('cnn+pylidc_features.csv', index=False)

Even after using Global Average Pooling the output of our CNN is 512 feature which is way too high. So we need to reduce it even more

In [2]:
import pandas as pd

df=pd.read_csv('cnn+pylidc_features.csv')
print(df.shape)
df.head()

(2625, 527)


,PatientID,Calcification,Lobulation,Spiculation,Internal Texture,Diameter (mm),Surface Area (mm^2),Volume (mm^3),Pixeis_in_lesion,Circularity,...,cnn_feature 503,cnn_feature 504,cnn_feature 505,cnn_feature 506,cnn_feature 507,cnn_feature 508,cnn_feature 509,cnn_feature 510,cnn_feature 511,Label
0,LIDC-IDRI-0001,6,3,4,5,32.755812,2491.466573,6989.673615,5428,0.121212,...,0.080818,1.799512,0.474278,6.975934,1.273983,4.503269,0.129298,9.106273,0.563464,5
1,LIDC-IDRI-0002,6,1,1,2,30.781671,2807.198994,7244.667508,14252,0.243756,...,3.473532,0.753911,0.949201,1.776874,2.387758,4.194271,1.476390,5.335127,1.748041,4
2,LIDC-IDRI-0003,6,1,1,1,31.664468,1996.252117,4731.410934,2542,0.154584,...,0.801210,0.239504,9.201927,1.946184,5.459285,0.320320,0.616315,3.036831,0.429381,2
3,LIDC-IDRI-0003,6,2,3,4,31.001964,2225.677350,6519.463698,3241,0.139765,...,0.951502,1.994685,0.370287,5.199282,1.392501,1.789750,0.671943,5.783095,0.068551,4
4,LIDC-IDRI-0003,6,2,2,5,13.309155,321.183599,472.089669,261,0.271896,...,0.095775,1.139045,0.818399,4.912726,11.126127,1.109776,1.508004,11.945731,0.484690,3


In order to make the randomforest algorithm to work better we decided to make the number of features smaller 200 and 50, using PCA. Which basicly is going to choose the principal componets having into account their variance and covariance to understand if certain features overlap and which ones represent more overall variance 

In [8]:
from sklearn.decomposition import PCA
pca=PCA(n_components=50)
features_columns=df.loc[:,~df.columns.isin(['PatientID','Label',"Calcification",
              "Lobulation",
              "Spiculation",
              "Internal Texture",
              "Diameter (mm)",
              "Surface Area (mm^2)",
              "Volume (mm^3)",
              "Pixeis_in_lesion",
              "Circularity",
              "Median_HU_in_lesion",
              "STD_HU_in_lesion",
              "VAR_HU_in_lession",
              "Mean_HU_in_lesion"])]
reduced_features=pca.fit_transform(features_columns)
reduced_features.shape

(2625, 50)

Formating the reduced features into an DataFrame

In [ ]:
columns_name=[]
for i in range(50):
    columns_name.append("cnn_feature "+str(i))
    
reduced_df=pd.DataFrame(columns=columns_name, data=reduced_features)
reduced_df=pd.concat([df[['PatientID',"Calcification",
              "Lobulation",
              "Spiculation",
              "Internal Texture",
              "Diameter (mm)",
              "Surface Area (mm^2)",
              "Volume (mm^3)",
              "Pixeis_in_lesion",
              "Circularity",
              "Median_HU_in_lesion",
              "STD_HU_in_lesion",
              "VAR_HU_in_lession",
              "Mean_HU_in_lesion"]],reduced_df,df['Label']], axis=1)
reduced_df.head()

,PatientID,Calcification,Lobulation,Spiculation,Internal Texture,Diameter (mm),Surface Area (mm^2),Volume (mm^3),Pixeis_in_lesion,Circularity,...,cnn_feature 41,cnn_feature 42,cnn_feature 43,cnn_feature 44,cnn_feature 45,cnn_feature 46,cnn_feature 47,cnn_feature 48,cnn_feature 49,Label
0,LIDC-IDRI-0001,6,3,4,5,32.755812,2491.466573,6989.673615,5428,0.121212,...,7.040247,-1.513496,0.775723,-0.234806,-5.807555,-3.458864,-4.258366,4.971976,-1.937084,5
1,LIDC-IDRI-0002,6,1,1,2,30.781671,2807.198994,7244.667508,14252,0.243756,...,3.515642,5.453791,-4.427953,-4.131188,-2.238985,-0.942886,-7.291642,7.063912,4.522726,4
2,LIDC-IDRI-0003,6,1,1,1,31.664468,1996.252117,4731.410934,2542,0.154584,...,5.648545,-5.778849,-2.962466,6.492604,-3.243713,-0.093044,-7.528972,1.491716,-5.227047,2
3,LIDC-IDRI-0003,6,2,3,4,31.001964,2225.677350,6519.463698,3241,0.139765,...,-0.395001,-0.695193,-0.807704,1.977416,-2.345557,-1.399320,-3.947806,9.112702,-2.307368,4
4,LIDC-IDRI-0003,6,2,2,5,13.309155,321.183599,472.089669,261,0.271896,...,-1.828549,-1.749191,-3.188446,-0.763180,0.101668,-0.589906,1.837360,-0.556560,-2.447910,3


In [11]:
reduced_df.shape

(2625, 65)

Store the reduced dataset for easy access to both the full set of features and the reduced

In [12]:
#storing the reduced cnn_features
reduced_df.to_csv('cnn_reduced_50+pylidc_features.csv',index=False)

For comparation sake we created an dataset without the cnn features to understand if they are ginving new information to our preditive model

In [15]:
#create a dataset without the cnn features
pylidc_df=df[['PatientID',"Calcification",
              "Lobulation",
              "Spiculation",
              "Internal Texture",
              "Diameter (mm)",
              "Surface Area (mm^2)",
              "Volume (mm^3)",
              "Pixeis_in_lesion",
              "Circularity",
              "Median_HU_in_lesion",
              "STD_HU_in_lesion",
              "VAR_HU_in_lession",
              "Mean_HU_in_lesion","Label"]]

print(pylidc_df.shape)
pylidc_df.head()


(2625, 15)


,PatientID,Calcification,Lobulation,Spiculation,Internal Texture,Diameter (mm),Surface Area (mm^2),Volume (mm^3),Pixeis_in_lesion,Circularity,Median_HU_in_lesion,STD_HU_in_lesion,VAR_HU_in_lession,Mean_HU_in_lesion,Label
0,LIDC-IDRI-0001,6,3,4,5,32.755812,2491.466573,6989.673615,5428,0.121212,24.0,224.079908,50211.805256,-77.425571,5
1,LIDC-IDRI-0002,6,1,1,2,30.781671,2807.198994,7244.667508,14252,0.243756,-725.0,133.215507,17746.371374,-703.924993,4
2,LIDC-IDRI-0003,6,1,1,1,31.664468,1996.252117,4731.410934,2542,0.154584,-660.0,142.814992,20396.121845,-640.765932,2
3,LIDC-IDRI-0003,6,2,3,4,31.001964,2225.677350,6519.463698,3241,0.139765,-104.0,218.552088,47765.015033,-179.850663,4
4,LIDC-IDRI-0003,6,2,2,5,13.309155,321.183599,472.089669,261,0.271896,-405.0,232.023157,53834.745526,-359.831418,3


Store it for easier access

In [16]:
#store the dataset without cnn features
pylidc_df.to_csv("pylidc_features.csv", index=False)

Now we wnat to map the Labels because having 5 classes is not realy necessary, for example if an nodule is an 1 or an 2 it still is not considered Suspicious by radiologist. Other mappings also appeared to make sense like an nodule is either suspicous, not suspicous or we don't have enough infomation. Due to the fact that the class 3 considerated not enough information and the class 2 moderatly not suspicous are the most common one by far, and also because our predimitve model seemed to have problems distinguishing  both of them we decided we could put the class 2 inside the class 3 as if it is moderalty not suspicous there is no danger in cosider it not enough information because no agressive treatments would be prescribed either way and if anything it being class 3 it would just be put on an whatch list to be further analysed another time. Another idea to that problem was only using tow classes 1 and 5, however since the class 3 was a big part of our dataset we didn't want to simply not used and as such we created two label one in which the class 3 would be inside the class 1 (an conservative aproach because if we don't have enough information is better to say that it is not suspicious and avoid agreesive treatments) and one where the class 3 is inside the class 5(more agressive approach since it would say suspicous if it is in "doubt")

In [ ]:
df=pd.read_csv('datasets/cnn_reduced_50+pylidc_features.csv')
#define the mapping desired
# mapping 1
#label_mapping= {1:1, 2:1, 3:3, 4:5, 5:5}
# mapping 2
#label_mapping= {1:1, 2:3, 3:3, 4:5, 5:5}
# mapping 3
#label_mapping= {1:1, 2:1, 3:5, 4:5, 5:5}
# mapping 4
label_mapping= {1:1, 2:1, 3:1, 4:5, 5:5}


#aplying it 
df['Label']=df['Label'].map(label_mapping)

df.to_csv('datasets/cnn_reduced_50+pylidc_features_nl4.csv', index= False)